In [26]:
# Importing necessary libraries
from ipywidgets import VBox, HBox
import ipywidgets as widgets
from IPython.display import clear_output
from PIL import Image
import sys
from IPython.display import HTML
from pathlib import Path
from io import BytesIO
import json
import os
import requests as requests
from IPython.display import HTML
from deepface import DeepFace
from deepface.extendedmodels import  Gender
import os
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from zipfile import ZipFile
import tarfile
from urllib.request import urlopen
from scipy.io import loadmat
import csv
import sys
import re


## Load the gender model from deepface
models = {}
models["gender"] = Gender.loadModel()

In [27]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [28]:
#running deepface
def deepface(filename):
    deepface_pred = DeepFace.analyze(f"images/{filename}", actions=['gender'] ,models=models,enforce_detection= True)
    return deepface_pred

In [29]:
'''Set up layout for upload widgets'''
def initialize_widgets():
    global upload, layout
    upload = widgets.FileUpload(
        accept='image/*', 
        multiple=True  
    )
    layout = VBox([upload, submit])
    display(layout)
    
global clear, submit, submitted
submitted = False
submit = widgets.Button(description="Submit")
clear = widgets.Button(description="Clear")
initialize_widgets()

''' Clear the output and delete the files'''
def on_clear_clicked(b):
    global submitted
    clear_output()
    initialize_widgets()
    try:
        shutil.rmtree('./images/')
    except:
        pass
    for filename in os.listdir('./images/'):
        if filename.endswith(".csv"):
            os.remove(filename)
    submitted = False

'''Resize and convert images for M3. Save the file (is deleted afterwards)'''    
def preprocess_and_save(content, filename):
    stream = BytesIO(content)
    image = Image.open(stream).convert("RGB")
    stream.close()
    image.save(f"images/{filename}")

'''Submitting an image either as fileupload or from url'''
def on_submit_clicked(b):
    global submitted, content
    if not upload.value: #no input is given
        print('No image has been uploaded')
        return False
    Path("images/").mkdir(parents=True, exist_ok=True)
    with open('output.csv', 'w', newline='') as output:  # output file is created
        wr = csv.writer(output,quoting=csv.QUOTE_ALL)
        wr.writerow(['Filename','Gender'])
        for filename in upload.value.keys():  
            content = upload.value[filename]['content'] #extracting the bytes for images
            preprocess_and_save(content, filename)
            try:
                pred_gender=deepface(filename)
                wr = csv.writer(output,quoting=csv.QUOTE_ALL)
                wr.writerow([filename,pred_gender['gender']])
            except:
                wr = csv.writer(output,quoting=csv.QUOTE_ALL)
                wr.writerow([filename,'NaN'])   #if there is an error in returning gender , write undefined
            #print("Oops!",sys.exc_info()[0],"occured.")
    if submitted: #cleaning the previous results
        clear_output()
        initialize_widgets()
    submitted = True
    output_df = pd.read_csv("output.csv")
    print("The results are presented in the table below. NaN values indicate that no faces have been detected on a picture.")
    display(output_df, clear)

submit.on_click(on_submit_clicked)
clear.on_click(on_clear_clicked)


Action: gender:   0%|          | 0/1 [00:00<?, ?it/s] 

Actions to do:  ['gender']
already built gender model is passed



Action: gender:   0%|          | 0/1 [00:00<?, ?it/s] 

Actions to do:  ['gender']
already built gender model is passed
Actions to do:  ['gender']
already built gender model is passed



Action: gender:   0%|          | 0/1 [00:00<?, ?it/s] 

Actions to do:  ['gender']
already built gender model is passed



Analyzing:   0%|          | 0/1 [00:00<?, ?it/s]

The results are presented in the table below. NaN values indicate that no faces have been detected on a picture.


,Filename,Gender
0,Abdulrahman+M.+Al-Ahmari.jpg,Man
1,Abu+James+Sundufu.jpg,NaN
2,Acaio+Zuniga+Leite.jpg,Man
3,Adam+M.+Sonabend.jpg,Man


Button(description='Clear', style=ButtonStyle())

In [30]:
text = HTML('<p>The results are saved in the <a href="output.csv" target="_blank">csv file</a> and the folder with photos is <a href="images" target="_blank">here</a> (in case you did not press on "Clear" yet, which deletes the stored data). </p>')
display(text)